In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import linear_model

In [6]:
# Read all the data
building1 = pd.read_csv("../data/schemas/warm_up/Building_1.csv", sep=",")
building2 = pd.read_csv("../data/schemas/warm_up/Building_2.csv", sep=",")
building3 = pd.read_csv("../data/schemas/warm_up/Building_3.csv", sep=",")
weather = pd.read_csv("../data/schemas/warm_up/weather.csv", sep=",")
carbon_intensity = pd.read_csv("../data/schemas/warm_up/carbon_intensity.csv", sep=",")

# Distinguish between the buildings
building1 = building1.add_suffix("_1")
building2 = building2.add_suffix("_2")
building3 = building3.add_suffix("_3")

# Merge
data = [building1, building2, building3, weather, carbon_intensity]
merged = pd.concat(data, axis=1)
merged.head()

,Month_1,Hour_1,Day Type_1,Daylight Savings Status_1,Indoor Temperature (C)_1,Average Unmet Cooling Setpoint Difference (C)_1,Indoor Relative Humidity (%)_1,Equipment Electric Power (kWh)_1,DHW Heating (kWh)_1,Cooling Load (kWh)_1,...,6h Outdoor Relative Humidity (%),12h Outdoor Relative Humidity (%),24h Outdoor Relative Humidity (%),6h Diffuse Solar Radiation (W/m2),12h Diffuse Solar Radiation (W/m2),24h Diffuse Solar Radiation (W/m2),6h Direct Solar Radiation (W/m2),12h Direct Solar Radiation (W/m2),24h Direct Solar Radiation (W/m2),kg_CO2/kWh
0,6,1,5,0,23.098652,-0.123570,61.086187,0.356839,0.055682,1.119216,...,72.980273,41.822361,83.230997,54.625927,116.842886,0.0,143.324335,1020.756093,0.0,0.402488
1,6,2,5,0,22.234742,0.012520,64.361378,0.345078,0.159338,1.469638,...,87.317655,37.910023,95.353948,78.375479,89.220504,0.0,300.361061,825.097292,0.0,0.382625
2,6,3,5,0,22.223061,0.000838,64.493415,0.338769,0.057004,1.458372,...,60.341453,30.190199,66.266409,259.822134,82.855932,0.0,196.691127,987.909330,0.0,0.369458
3,6,4,5,0,22.222251,0.000029,64.769702,0.334856,0.000000,1.337342,...,50.063925,31.519739,94.456370,248.631525,105.130119,0.0,465.483196,854.331825,0.0,0.367017
4,6,5,5,0,22.222236,0.000014,65.152120,0.348607,0.000000,1.163453,...,40.942579,33.119543,100.000000,272.825159,75.627986,0.0,569.203332,941.243325,0.0,0.374040


In [7]:
merged_dropped = merged.drop(columns=["Month_1", "Month_2", "Month_3",
                                      "Daylight Savings Status_1", "Daylight Savings Status_2", "Daylight Savings Status_3",
                                      "Heating Load (kWh)_1", "Heating Load (kWh)_2", "Heating Load (kWh)_3",
                                      "HVAC Mode (Off/Cooling/Heating)_1", "HVAC Mode (Off/Cooling/Heating)_2", "HVAC Mode (Off/Cooling/Heating)_3"])

# Insert an index column
index = range(1, len(merged_dropped)+1)
merged_dropped.insert(loc=0, column="Index", value=index)

# Add a column with average solar generation for later prediction
merged_dropped["Avg solar generation"] = merged_dropped[["Solar Generation (W/kW)_1", "Solar Generation (W/kW)_2", "Solar Generation (W/kW)_3"]].mean(axis=1)

In [10]:
total_length = merged_dropped.shape[0]

SIZE_OF_TEST = 24

train = merged_dropped.head(total_length - SIZE_OF_TEST)
test = merged_dropped.tail(SIZE_OF_TEST)

columns = list(merged_dropped)
columns.remove("Index")

for column in columns:
    min_of_column = train[column].min()
    max_of_column = train[column].max()
    train[column] = (train[column] - min_of_column) / (max_of_column - min_of_column)
    test[column] = (test[column] - min_of_column) / (max_of_column - min_of_column)

/tmp/ipykernel_32210/1094962155.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[column] = (train[column] - min_of_column) / (max_of_column - min_of_column)
/tmp/ipykernel_32210/1094962155.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[column] = (test[column] - min_of_column) / (max_of_column - min_of_column)
/tmp/ipykernel_32210/1094962155.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [12]:
test.head()

,Index,Hour_1,Day Type_1,Indoor Temperature (C)_1,Average Unmet Cooling Setpoint Difference (C)_1,Indoor Relative Humidity (%)_1,Equipment Electric Power (kWh)_1,DHW Heating (kWh)_1,Cooling Load (kWh)_1,Solar Generation (W/kW)_1,...,12h Outdoor Relative Humidity (%),24h Outdoor Relative Humidity (%),6h Diffuse Solar Radiation (W/m2),12h Diffuse Solar Radiation (W/m2),24h Diffuse Solar Radiation (W/m2),6h Direct Solar Radiation (W/m2),12h Direct Solar Radiation (W/m2),24h Direct Solar Radiation (W/m2),kg_CO2/kWh,Avg solar generation
696,697,0.000000,0.833333,0.459604,0.379385,0.550397,0.004843,0.0,0.236572,0.0,...,0.020023,0.671326,0.214351,0.405694,0.0,0.075161,0.665597,0.0,0.517775,0.0
697,698,0.043478,0.833333,0.515255,0.401406,0.510064,0.004637,0.0,0.145307,0.0,...,0.019901,0.853475,0.267622,0.671206,0.0,0.310976,0.394174,0.0,0.563846,0.0
698,699,0.086957,0.833333,0.614527,0.392546,0.423027,0.004455,0.0,0.075989,0.0,...,-0.004999,0.924014,0.432290,0.936496,0.0,0.515621,0.474656,0.0,0.569727,0.0
699,700,0.130435,0.833333,0.615362,0.402729,0.473087,0.004264,0.0,0.077754,0.0,...,0.005910,0.666555,0.450985,0.591760,0.0,0.540567,0.270914,0.0,0.582340,0.0
700,701,0.173913,0.833333,0.615388,0.403044,0.481986,0.004058,0.0,0.084912,0.0,...,0.028453,1.000000,0.364305,0.624536,0.0,0.682238,0.295213,0.0,0.557425,0.0


In [3]:
threshold = 0
column_to_predict = "kg_CO2/kWh"

independent_variables = []


for column in train:
    if abs(train[column].corr(train[column_to_predict])) > threshold:
        independent_variables.append(column)

independent_variables.remove(column_to_predict)
independent_variables

['Index',
 'Hour_1',
 'Day Type_1',
 'Indoor Temperature (C)_1',
 'Average Unmet Cooling Setpoint Difference (C)_1',
 'Indoor Relative Humidity (%)_1',
 'Equipment Electric Power (kWh)_1',
 'DHW Heating (kWh)_1',
 'Cooling Load (kWh)_1',
 'Solar Generation (W/kW)_1',
 'Occupant Count (people)_1',
 'Temperature Set Point (C)_1',
 'Hour_2',
 'Day Type_2',
 'Indoor Temperature (C)_2',
 'Average Unmet Cooling Setpoint Difference (C)_2',
 'Indoor Relative Humidity (%)_2',
 'Equipment Electric Power (kWh)_2',
 'DHW Heating (kWh)_2',
 'Cooling Load (kWh)_2',
 'Solar Generation (W/kW)_2',
 'Occupant Count (people)_2',
 'Temperature Set Point (C)_2',
 'Hour_3',
 'Day Type_3',
 'Indoor Temperature (C)_3',
 'Average Unmet Cooling Setpoint Difference (C)_3',
 'Indoor Relative Humidity (%)_3',
 'Equipment Electric Power (kWh)_3',
 'DHW Heating (kWh)_3',
 'Cooling Load (kWh)_3',
 'Solar Generation (W/kW)_3',
 'Occupant Count (people)_3',
 'Temperature Set Point (C)_3',
 'Outdoor Drybulb Temperature 

In [5]:
train.shape

(684, 52)